In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

In [ ]:
train_set=pd.read_csv('/content/drive/MyDrive/CoLi-Dravidian_2024/mal_train.csv')
val_set=pd.read_csv('/content/drive/MyDrive/CoLi-Dravidian_2024/mal_val.csv')

In [ ]:
train_set.head()

,Word,Tag
0,Supr,ENGLISH
1,trailer,ENGLISH
2,oru,MALAYALAM
3,vettame,MALAYALAM
4,like,ENGLISH


In [ ]:
!pip install word2number
!pip install inflect
import pandas as pd
import inflect
from word2number import w2n

p = inflect.engine()

def number_to_words(number_text):
    try:
        number_value = w2n.word_to_num(number_text)
        word_format = p.number_to_words(number_value)
        return word_format
    except ValueError:
        return number_text


df_train = pd.DataFrame(train_set)
df_val=pd.DataFrame(val_set)

df_train['Word'] = df_train.apply(lambda row: number_to_words(row['Word']) if row['Tag'] == 'NUMBER' else row['Word'], axis=1)
df_val['Word'] = df_val.apply(lambda row: number_to_words(row['Word']) if row['Tag'] == 'NUMBER' else row['Word'], axis=1)

In [ ]:
df_train.to_csv('/content/drive/MyDrive/CoLi-Dravidian_2024/preprocess_num_word_train.csv')
df_val.to_csv('/content/drive/MyDrive/CoLi-Dravidian_2024/preprocess_num_word_val.csv')

**TF-IDF with Romanized Char [ TF-IDF of char n-grams of range (1,5) ]**

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,5), analyzer= 'char')
X = vectorizer.fit(df_train['Word'])
train_tfidf=vectorizer.transform(df_train['Word'])
dev_tfidf=vectorizer.transform(df_val['Word'])

**ML Model Building : SVM, LR, MNB, DT, CatBoost, and Ensemble of ML classifiers trained with romanized char TF-IDF**

In [ ]:
clf1= svm.SVC()
clf1.fit(train_tfidf, df_train['Tag'])
val_pred1=clf1.predict(dev_tfidf)

In [ ]:
print(classification_report(df_val['Tag'], val_pred1))

In [ ]:
clf2=LogisticRegression(solver='liblinear', multi_class='ovr')
clf2.fit(train_tfidf, df_train['Tag'])
val_pred2=clf2.predict(dev_tfidf)

In [ ]:
print(classification_report(df_val['Tag'], val_pred2))

In [ ]:
clf3 = MultinomialNB(force_alpha=True)
clf3.fit(train_tfidf, df_train['Tag'])
val_pred3=clf3.predict(dev_tfidf)

In [ ]:
print(classification_report(df_val['Tag'], val_pred3))

In [ ]:
clf4 = DecisionTreeClassifier(random_state=0)
clf4.fit(train_tfidf, df_train['Tag'])
val_pred4=clf4.predict(dev_tfidf)

In [ ]:
print(classification_report(df_val['Tag'], val_pred4))

In [ ]:
clf5 = KNeighborsClassifier(n_neighbors=3)
clf5.fit(train_tfidf, df_train['Tag'])
val_pred5=clf5.predict(dev_tfidf)

In [ ]:
print(classification_report(df_val['Tag'], val_pred5))

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=1000,  # Number of trees (default is 1000)
                           learning_rate=0.1,  # Learning rate (default is 0.03)
                           depth=6,  # Depth of the trees (default is 6)
                           loss_function='MultiClass',  # Loss function (default is 'Logloss')
                           verbose=100  # Controls the level of verbosity (default is 100)
                           )

# Train the model
model.fit(train_tfidf, df_train['Tag'], eval_set=(dev_tfidf, df_val['Tag']), early_stopping_rounds=50, verbose=100)

# Predictions on the val set
y_pred = model.predict(dev_tfidf)

print("\n", classification_report(df_val['Tag'], y_pred))

In [ ]:
voting = VotingClassifier(
    estimators=[('SVM', clf2), ('DecisionTreeClassifier', clf4),('kNN', clf5)],
    voting='hard')

In [ ]:
voting.fit(train_tfidf, df_train['Tag'])

VotingClassifier(estimators=[('SVM',
                              LogisticRegression(multi_class='ovr',
                                                 solver='liblinear')),
                             ('DecisionTreeClassifier',
                              DecisionTreeClassifier(random_state=0)),
                             ('kNN', KNeighborsClassifier(n_neighbors=3))])

In [ ]:
voting_pred= voting.predict(dev_tfidf)
print(classification_report(df_val['Tag'], voting_pred))

In [ ]:
#Submission
# to create .csv file consisting of Word and Tag as two columns
y_pred = pd.DataFrame(data=val_pred1, columns=['final_prediction'])
Submisssion_mal= pd.DataFrame()
Submisssion_mal['Word'] = val_set['Word']
Submisssion_mal['Tag'] = y_pred
Submisssion_mal.to_csv('/content/drive/MyDrive/CoLi-Dravidian_2024/predictions.csv',index = None)